In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [ ]:
%cd /content/drive/MyDrive/PRBNet_PyTorch_Final/prb

/content/drive/MyDrive/PRBNet_PyTorch_Final/prb


In [ ]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 39.4 MB/s 


In [ ]:
# import
from copy import deepcopy
from models.yolo import Model
import torch
from utils.torch_utils import select_device, is_parallel
import yaml

cfg_path = r"cfg/training/PRB_Series/yolov7-PRB.yaml"
ckpt_path = r"/content/drive/MyDrive/PRBNet_PyTorch/prb/runs/train/yolov7-prb-custom20/weights/best.pt"  # checkpoints of yolov7-prb with IDetect layer
save_path = r"yolov7-prb_final.pt"  # output file path

device = select_device('0', batch_size=1)
# model trained by cfg/training/*.yaml
ckpt = torch.load(ckpt_path, map_location=device)
# reparameterized model in cfg/deploy/*.yaml
model = Model(cfg_path, ch=3, nc=4).to(device)

with open(cfg_path) as f:
    yml = yaml.load(f, Loader=yaml.SafeLoader)
anchors = len(yml['anchors'])

# copy intersect weights
state_dict = ckpt['model'].float().state_dict()
exclude = []
intersect_state_dict = {k: v for k, v in state_dict.items() if k in model.state_dict() and not any(x in k for x in exclude) and v.shape == model.state_dict()[k].shape}
model.load_state_dict(intersect_state_dict, strict=False)
model.names = ckpt['model'].names
model.nc = ckpt['model'].nc

# reparametrized YOLOR
for i in range((model.nc+5)*anchors):  
    model.state_dict()['model.108.m.0.weight'].data[i, :, :, :] *= state_dict['model.108.im.0.implicit'].data[:, i, : :].squeeze()
    model.state_dict()['model.108.m.1.weight'].data[i, :, :, :] *= state_dict['model.108.im.1.implicit'].data[:, i, : :].squeeze()
    model.state_dict()['model.108.m.2.weight'].data[i, :, :, :] *= state_dict['model.108.im.2.implicit'].data[:, i, : :].squeeze()
model.state_dict()['model.108.m.0.bias'].data += state_dict['model.108.m.0.weight'].mul(state_dict['model.108.ia.0.implicit']).sum(1).squeeze()
model.state_dict()['model.108.m.1.bias'].data += state_dict['model.108.m.1.weight'].mul(state_dict['model.108.ia.1.implicit']).sum(1).squeeze()
model.state_dict()['model.108.m.2.bias'].data += state_dict['model.108.m.2.weight'].mul(state_dict['model.108.ia.2.implicit']).sum(1).squeeze()
model.state_dict()['model.108.m.0.bias'].data *= state_dict['model.108.im.0.implicit'].data.squeeze()
model.state_dict()['model.108.m.1.bias'].data *= state_dict['model.108.im.1.implicit'].data.squeeze()
model.state_dict()['model.108.m.2.bias'].data *= state_dict['model.108.im.2.implicit'].data.squeeze()

# model to be saved
ckpt = {'model': deepcopy(model.module if is_parallel(model) else model).half(),
        'optimizer': None,
        'training_results': None,
        'epoch': -1}

# save reparameterized model
torch.save(ckpt, save_path)

In [ ]:
# finetune yolov7-prb models
!python train.py --batch-size 2 --data data/TWStreet.yaml --epochs 600 --img 1920 --cfg cfg/training/PRB_Series/yolov7-PRB.yaml --weights 'yolov7-prb_final.pt' --name yolov7-prb-custom --hyp data/hyp.scratch.custom.yaml

YOLOR 🚀 4d5304c torch 1.12.1+cu113 CUDA:0 (A100-SXM4-40GB, 40536.1875MB)

Namespace(adam=False, artifact_alias='latest', batch_size=2, bbox_interval=-1, bucket='', cache_images=False, cfg='cfg/training/PRB_Series/yolov7-PRB.yaml', data='data/TWStreet.yaml', device='', entity=None, epochs=600, evolve=False, exist_ok=False, freeze=[0], global_rank=-1, hyp='data/hyp.scratch.custom.yaml', image_weights=False, img_size=[1920, 1920], label_smoothing=0.0, linear_lr=False, local_rank=-1, multi_scale=False, name='yolov7-prb-custom', noautoanchor=False, nosave=False, notest=False, project='runs/train', quad=False, rect=False, resume=False, save_dir='runs/train/yolov7-prb-custom28', save_period=-1, single_cls=False, sync_bn=False, total_batch_size=2, upload_dataset=False, v5_metric=False, weights='yolov7-prb_final.pt', workers=8, world_size=1)
tensorboard: Start with 'tensorboard --logdir runs/train', view at http://localhost:6006/
2022-11-23 04:58:05.830943: I tensorflow/core/util/util.cc:169] o

In [ ]:
# import
from copy import deepcopy
from models.yolo import Model
import torch
from utils.torch_utils import select_device, is_parallel
import yaml

cfg_path = r"cfg/deploy/yolov7-PRB.yaml"
ckpt_path = r"/content/drive/MyDrive/PRBNet_PyTorch/prb/runs/train/yolov7-prb-custom28/weights/best.pt"  # checkpoints of yolov7-prb with IDetect layer
save_path = r"best_1920.pt"  # output file path

device = select_device('0', batch_size=1)
# model trained by cfg/training/*.yaml
ckpt = torch.load(ckpt_path, map_location=device)
# reparameterized model in cfg/deploy/*.yaml
model = Model(cfg_path, ch=3, nc=4).to(device)

with open(cfg_path) as f:
    yml = yaml.load(f, Loader=yaml.SafeLoader)
anchors = len(yml['anchors'])

# copy intersect weights
state_dict = ckpt['model'].float().state_dict()
exclude = []
intersect_state_dict = {k: v for k, v in state_dict.items() if k in model.state_dict() and not any(x in k for x in exclude) and v.shape == model.state_dict()[k].shape}
model.load_state_dict(intersect_state_dict, strict=False)
model.names = ckpt['model'].names
model.nc = ckpt['model'].nc

# reparametrized YOLOR
for i in range((model.nc+5)*anchors):  
    model.state_dict()['model.108.m.0.weight'].data[i, :, :, :] *= state_dict['model.108.im.0.implicit'].data[:, i, : :].squeeze()
    model.state_dict()['model.108.m.1.weight'].data[i, :, :, :] *= state_dict['model.108.im.1.implicit'].data[:, i, : :].squeeze()
    model.state_dict()['model.108.m.2.weight'].data[i, :, :, :] *= state_dict['model.108.im.2.implicit'].data[:, i, : :].squeeze()
model.state_dict()['model.108.m.0.bias'].data += state_dict['model.108.m.0.weight'].mul(state_dict['model.108.ia.0.implicit']).sum(1).squeeze()
model.state_dict()['model.108.m.1.bias'].data += state_dict['model.108.m.1.weight'].mul(state_dict['model.108.ia.1.implicit']).sum(1).squeeze()
model.state_dict()['model.108.m.2.bias'].data += state_dict['model.108.m.2.weight'].mul(state_dict['model.108.ia.2.implicit']).sum(1).squeeze()
model.state_dict()['model.108.m.0.bias'].data *= state_dict['model.108.im.0.implicit'].data.squeeze()
model.state_dict()['model.108.m.1.bias'].data *= state_dict['model.108.im.1.implicit'].data.squeeze()
model.state_dict()['model.108.m.2.bias'].data *= state_dict['model.108.im.2.implicit'].data.squeeze()

# model to be saved
ckpt = {'model': deepcopy(model.module if is_parallel(model) else model).half(),
        'optimizer': None,
        'training_results': None,
        'epoch': -1}

# save reparameterized model
torch.save(ckpt, save_path)

/usr/local/lib/python3.7/dist-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [ ]:
!python try.py --weights best_1920.pt --img 1920 --source /content/drive/MyDrive/public --augment --save-txt

Namespace(agnostic_nms=False, augment=True, classes=None, conf_thres=0.25, device='', exist_ok=False, img_size=1920, iou_thres=0.45, name='exp', no_trace=False, nosave=False, project='runs/detect', save_conf=False, save_txt=True, source='/content/drive/MyDrive/public', update=False, view_img=False, weights=['best_1920.pt'])
YOLOR 🚀 4d5304c torch 1.12.1+cu113 CUDA:0 (Tesla T4, 15109.75MB)

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
/usr/local/lib/python3.7/dist-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 887 layers, 100706449 parameters, 12436561 gradients, 251.5 GFLOPS
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

30 cars, 5 hovs, 1 person, 6 moto